In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.colors import LogNorm

from astropy.table import Table, join, QTable, vstack
import astropy.units as u
import sys
import pyneb as pn
from multiprocessing import Pool
import multiprocessing as mp
import math
from astropy.io import fits
from orcs.process import SpectralCube

from astropy.nddata import NDData, Cutout2D
from astropy.wcs import WCS

import astropy
import astropy.units as u
from astropy.coordinates import SkyCoord

from reproject import reproject_interp
import reproject
from regions import PixCoord

import pylab as pl

from scipy.optimize import curve_fit
from scipy.integrate import quad

from orb.fit import fit_lines_in_spectrum
from orb.utils.spectrum import corr2theta, amp_ratio_from_flux_ratio
from orb.core import Lines
import gvar
import orb

import extinction
from extinction import apply, remove

from photutils.detection import DAOStarFinder

import aplpy

from astropy.stats import sigma_clipped_stats
from photutils.psf import extract_stars
from astropy.visualization import simple_norm
from photutils.psf import EPSFBuilder

from mpl_toolkits.mplot3d import Axes3D

### Import data

In [ ]:
galaxynum = 6
galdic = {1:'NGC4254', 2:'NGC4535', 3:'NGC3351', 4:'NGC2835', 5:'NGC0628', 6:'NGC3627'}  #There is no SITELLE data for NGC 4254, NGC 2835 has the best data 
galaxy = galdic[galaxynum]
galaxy

galveldic = {'NGC4254': 2388 , 'NGC4535': 1954  , 'NGC3351': 775, 'NGC2835': 867, 'NGC0628':651, 'NGC3627':715}
galvel = galveldic[galaxy]

inter_data_path = '/home/habjan/SITELLE/data/data_raw_intermediate'

infile = inter_data_path + f"/{galaxy}_cube.hdf5"
cube = SpectralCube(infile)

hdul = fits.open(inter_data_path + f"/{galaxy}_IMAGE_FOV_Johnson_B_WCS_Pall_mad.fits")
muse_data = NDData(data=hdul['DATA'].data, mask=np.isnan(hdul['DATA'].data), meta=hdul['DATA'].header, wcs=WCS(hdul['DATA'].header)) 

hdul = fits.open(inter_data_path + f"/{galaxy}_SDSS_g.fits")
fits_rband = hdul['DATA']
muse_rband = NDData(data=hdul['DATA'].data, mask=np.isnan(hdul['DATA'].data), meta=hdul['DATA'].header, wcs=WCS(hdul['DATA'].header)) 

hdul = fits.open(inter_data_path + f"/{galaxy}_Johnson_Bcopt.fits")
fits_bband = hdul['DATA']
muse_bband = NDData(data=hdul['DATA'].data, mask=np.isnan(hdul['DATA'].data), meta=hdul['DATA'].header, wcs=WCS(hdul['DATA'].header)) 

hdul = fits.open(inter_data_path + f"/{galaxy}_MAPS.fits")
Halpha = NDData(data=hdul['HA6562_FLUX'].data, mask=np.isnan(hdul['HA6562_FLUX'].data), meta=hdul['HA6562_FLUX'].header, wcs=WCS(hdul['HA6562_FLUX'].header))
Halpha.data[muse_data.data==0] = np.nan

hdul = fits.open(inter_data_path + f"/{galaxy}_nebulae_mask_V2.fits")
nebulae_mask = NDData(data = hdul[0].data.astype(float), mask=Halpha.mask, meta=hdul[0].header, wcs=WCS(hdul[0].header)) 
nebulae_mask.data[nebulae_mask.data==-1] = np.nan

hdul = fits.open(inter_data_path + f"/{galaxy}_cube.fits")
header = hdul[0].header
wcs = WCS(header,naxis=2)

infile = open(inter_data_path + "/Nebulae_catalogue_v3.fits",'rb')
hdul = Table.read(infile)
musedata = hdul[hdul['gal_name'] == f'{galaxy}']

hdul = fits.open(inter_data_path + f"/{galaxy}_deepframe.fits")
sit_deep = hdul[0]

coord_dic = {'NGC4535':np.array([188.5851585 , 8.19257]), 'NGC3351':np.array([160.99236896, 11.70541767]), 
             'NGC2835':np.array([139.47045857, -22.35414826]), 'NGC0628':np.array([24.17123567, 15.78081634]), 
             'NGC3627':np.array([170.06252, 12.9915])}
zoom_dic = {'NGC4535':np.array([0.05, 0.05]), 'NGC3351':np.array([0.053, 0.053]), 
            'NGC2835':np.array([0.05, 0.05]), 'NGC0628':np.array([0.08, 0.08]),
            'NGC3627':np.array([0.04, 0.085])}
galaxy

### Plot the MUSE data using aplpy and overlay the SITELLE data contours

In [ ]:
gc = aplpy.FITSFigure(fits_bband)
gc.show_contour(sit_deep, colors='green')

gc.recenter(coord_dic[galaxy][0], coord_dic[galaxy][1], width=zoom_dic[galaxy][0], height=zoom_dic[galaxy][1])
gc.show_colorscale(cmap='gist_heat', vmin=np.quantile(fits_bband.data, 0.4), vmax=np.quantile(fits_bband.data, 0.99))
gc.set_title(f'MUSE deep frame for {galaxy}')

# Find point sources to compare between datasets

### Use the appropriate settings to find bright sources in the MUSE image

In [ ]:
daofind = DAOStarFinder(fwhm=1/0.2, threshold=np.quantile(muse_bband.data, 0.8))
muse_sources = daofind(muse_bband.data)
len(muse_sources)

### Use the plot blow to find an individual source

In [ ]:
gc = aplpy.FITSFigure(fits_bband)

source = 13
plt.scatter(muse_sources[source]['xcentroid'], muse_sources[source]['ycentroid'], linewidth=2, s=500, c='green', alpha=1)
plt.scatter(muse_sources['xcentroid'], muse_sources['ycentroid'], marker='o', facecolors='none', linewidth=2, s=100, edgecolors='blue', alpha=1)

gc.recenter(coord_dic[galaxy][0], coord_dic[galaxy][1], width=zoom_dic[galaxy][0], height=zoom_dic[galaxy][1])
gc.show_colorscale(cmap='gist_heat', vmin=np.quantile(fits_bband.data, 0.5), vmax=np.quantile(fits_bband.data, 0.99))
gc.set_title(f'MUSE deep frame for {galaxy}')
gc.add_grid()

### Find the brightest points that show up in both the MUSE and SITELLE images. Manually enter them in this dictionary

In [ ]:
muse_psf_dic= {'NGC4535':np.array([218]), 'NGC3351':np.array([0, 4, 16, 28]), 
               'NGC2835':np.array([29]), 'NGC0628':np.array([89, 104]),
               'NGC3627':np.array([3])}

muse_wcs_dic= {'NGC4535':np.array([3, 39, 114, 218]), 'NGC3351':np.array([14, 18, 47, 49, 58, 61]), 
               'NGC2835':np.array([4, 29, 38, 41, 55, 56, 63]), 'NGC0628':np.array([89, 92, 104, 132, 168, 188, 251, 274]),
               'NGC3627':np.array([2, 3, 4, 13, 29, 30, 57])}

muse_bright_ind = muse_wcs_dic[galaxy]
muse_bright_wcs = muse_sources[muse_bright_ind]

### Plot only the brightest sources from the MUSE image

In [ ]:
gc = aplpy.FITSFigure(fits_bband)

plt.scatter(muse_bright_wcs['xcentroid'], muse_bright_wcs['ycentroid'], marker='o', facecolors='none', linewidth=2, s=100, edgecolors='blue', alpha=1)

gc.recenter(coord_dic[galaxy][0], coord_dic[galaxy][1], width=zoom_dic[galaxy][0], height=zoom_dic[galaxy][1])
gc.show_colorscale(cmap='gist_heat', vmin=np.quantile(fits_bband.data, 0.01), vmax=np.quantile(fits_bband.data, 0.995))
gc.set_title(f'MUSE deep frame for {galaxy}')
gc.add_grid()

### Find all of the brightest sources in the SITELLE image

In [ ]:
daofind = DAOStarFinder(fwhm=1/0.321, threshold=np.quantile(sit_deep.data, 0.8))
sit_sources = daofind(sit_deep.data)
len(sit_sources)

### Plot an individual source found above 

In [ ]:
gc = aplpy.FITSFigure(sit_deep)

source = 159
plt.scatter(sit_sources['xcentroid'][source], sit_sources['ycentroid'][source], marker='o', linewidth=2, s=500, c='green', alpha=1)
plt.scatter(sit_sources['xcentroid'], sit_sources['ycentroid'], marker='o', facecolors='none', linewidth=2, s=100, edgecolors='blue', alpha=1)

#gc.recenter(coord_dic[galaxy][0], coord_dic[galaxy][1], width=zoom_dic[galaxy][0], height=zoom_dic[galaxy][1])
gc.show_colorscale(cmap='gist_heat', vmin=np.quantile(sit_deep.data, 0.5), vmax=np.quantile(sit_deep.data, 0.9995))
gc.set_title(f'SITELLE deep frame for {galaxy}')
gc.add_grid()

### Manually enter the SITELLE sources here

In [ ]:
sit_psf_dic = {'NGC4535':np.array([12, 45, 63]), 'NGC3351':np.array([0, 3, 5, 6, 7, 11, 21, 61, 62, 63, 64, 66, 67, 69]), 
               'NGC2835':np.array([13, 58, 69, 81, 82, 109, 113, 115, 129]), 'NGC0628':np.array([41, 86, 137]),
               'NGC3627':np.array([0, 3, 7, 9, 13, 53, 70, 152, 155, 157, 159])}

sit_wcs_dic = {'NGC4535':np.array([7, 14, 27, 45]), 'NGC3351':np.array([16, 19, 43, 44, 53, 56]), 
               'NGC2835':np.array([46, 58, 64, 65, 71, 72, 79]), 'NGC0628':np.array([43, 44, 51, 61, 70, 76, 93, 100]),
               'NGC3627':np.array([7, 9, 12, 31, 65, 66, 148])}


sit_bright_ind = sit_wcs_dic[galaxy]
sit_bright_wcs = sit_sources[sit_bright_ind]

### Plot the SITELLE sources

In [ ]:
gc = aplpy.FITSFigure(sit_deep)

plt.scatter(sit_bright_wcs['xcentroid'], sit_bright_wcs['ycentroid'], marker='o', facecolors='none', linewidth=2, s=100, edgecolors='blue', alpha=1)

gc.recenter(coord_dic[galaxy][0], coord_dic[galaxy][1], width=zoom_dic[galaxy][0], height=zoom_dic[galaxy][1])
gc.show_colorscale(cmap='gist_heat', vmin=np.quantile(sit_deep.data, 0.5), vmax=np.quantile(sit_deep.data, 0.9995))
gc.set_title(f'SITELLE deep frame for {galaxy}')
gc.add_grid()

### Convert both the MUSE and SITELLE bright sources into degree coordinates

In [ ]:
muse_world = muse_rband.wcs.pixel_to_world(muse_bright_wcs['xcentroid'], muse_bright_wcs['ycentroid'])
muse_ra, muse_dec = muse_world.ra.degree, muse_world.dec.degree

sit_world = cube.get_wcs().pixel_to_world(sit_bright_wcs['xcentroid'], sit_bright_wcs['ycentroid'])
sit_ra, sit_dec = sit_world.ra.degree, sit_world.dec.degree

arcdiff = np.average(np.sqrt((muse_ra - sit_ra)**2 + (muse_dec - sit_dec)**2) * 3600)
print(f'The average arcsecond difference between datasets is {round(arcdiff, 5)}')

### Plot these coordiantes on a plain scatter plot for comparision

In [ ]:
point_s = 10

plt.scatter(muse_ra, muse_dec, c='k', s=point_s, label='MUSE')
plt.scatter(sit_ra, sit_dec, c='red' , s=point_s, alpha=0.75, label='SITELLE')

plt.title(f'Bright source comparison of MUSE and SITELLE in {galaxy}')
plt.figtext(0.5, -0.03, f'The average arcsecond difference between datasets is {round(arcdiff, 5)}'r'$^{\prime\prime}$', ha="center", fontsize=10, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
plt.xlabel('RA')
plt.ylabel('Dec')
plt.legend()

### Put the MUSE pixels into SITELLE coordinates

In [ ]:
muse_xsit, muse_ysit = cube.get_wcs().world_to_pixel(muse_world)

### Compare the SITELLE coordinates with the MUSE-to_SITELLE converted bright sources

In [ ]:
point_s = 10

plt.scatter(muse_xsit, muse_ysit, c='k', s=point_s, label='MUSE')
plt.scatter(sit_bright_wcs['xcentroid'], sit_bright_wcs['ycentroid'], c='red' , s=point_s, alpha=0.75, label='SITELLE')

plt.title(f'Bright source pixel location in MUSE and SITELLE in {galaxy}')
#plt.figtext(0.5, -0.03, f'The average arcsecond difference between datasets is {round(arcdiff, 5)}'r'$^{\prime\prime}$', ha="center", fontsize=10, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
plt.xlabel('X centroid')
plt.ylabel('Y centroid')
plt.legend()

### Plot the differences in RA and dec

In [ ]:
point_s = 10

plt.scatter((muse_ra - sit_ra) * 3600, (muse_dec - sit_dec) * 3600, c='k', s=point_s)

plt.title(f'Bright source difference in {galaxy}')
#plt.figtext(0.5, -0.03, f'The average arcsecond difference between datasets is {round(arcdiff, 5)}'r'$^{\prime\prime}$', ha="center", fontsize=10, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
plt.xlabel('RA (arcsecond)')
plt.ylabel('Dec (arcsecond)')
plt.legend()

### Find the average difference in RA and dec

In [ ]:
ra_diff_ave = np.average(muse_ra - sit_ra) * 3600
dec_diff_ave = np.average(muse_dec - sit_dec) * 3600
print(f'In {galaxy} the average difference in RA is {round(ra_diff_ave, 7)}" and in declination is {round(dec_diff_ave, 7)}"')

### Find the mean difference in RA and dec

In [ ]:
ra_diff_med = np.median(muse_ra - sit_ra) * 3600
dec_diff_med = np.median(muse_dec - sit_dec) * 3600
print(f'In {galaxy} the median difference in RA is {round(ra_diff_med, 7)}" and in declination is {round(dec_diff_med, 7)}"')

### Save bright sources that will be used for WCS calibration

In [ ]:
muse_bright_wcs.write(inter_data_path + f'/{galaxy}_muse_wcs_sources.fits', overwrite=True) 
sit_bright_wcs.write(inter_data_path + f'/{galaxy}_sitelle_wcs_sources.fits', overwrite=True) 

# Create ePSFs

### Select the bright sources that will be used for the PSFs

In [ ]:
muse_bright_ind = muse_psf_dic[galaxy]
muse_bright_psf = muse_sources[muse_bright_ind]

sit_bright_ind = sit_psf_dic[galaxy]
sit_bright_psf = sit_sources[sit_bright_ind]

### Create background subtract NDData objcts

In [ ]:
mean_val_muse, median_val_muse, std_val_muse = sigma_clipped_stats(fits_bband.data, sigma=2.0)
mean_val_sit, median_val_sit, std_val_sit = sigma_clipped_stats(sit_deep.data, sigma=2.0)

back_sit = sit_deep.data - median_val_sit
back_muse = fits_bband.data - median_val_muse

NDsit = NDData(data=back_sit)
NDmuse = NDData(data=back_muse)

### Create tables containing only the pixel locations of each image

In [ ]:
stars_sit = Table()
stars_sit['x'] = sit_bright_psf['xcentroid']
stars_sit['y'] = sit_bright_psf['ycentroid'] 

stars_muse = Table()
stars_muse['x'] = muse_bright_psf['xcentroid']
stars_muse['y'] = muse_bright_psf['ycentroid']

### Extract the stars using photutils

In [ ]:
im_pix = 25
ex_sit = extract_stars(NDsit, stars_sit, size=im_pix) 
yi_sit, xi_sit = np.indices(ex_sit[0].data.shape)

ex_muse = extract_stars(NDmuse, stars_muse, size=im_pix*(0.321/0.2)) 
yi_muse, xi_muse = np.indices(ex_muse[0].data.shape)

### Plot one of the sources in 3D

In [ ]:
fig = plt.figure()
frame = fig.add_subplot(1,1,1, projection='3d', azim=-45, elev=30)
frame.plot_surface(xi_sit, yi_sit, ex_sit[0].data, cmap='jet')
frame.set_xlabel('X-pixel')
frame.set_ylabel('Y-pixel')
frame.set_zlabel('Flux Amplitude')
frame.set_title(f'3D SITELLE star in {galaxy}')
plt.show()

### Define the functional form of a 2D gaussian

In [ ]:
def f(x, y, A, x0, y0, sigma_x, sigma_y):         ### This is actually not used, the function in the next cell is what is used to fit the point sources 
    return A*np.exp(-(x-x0)**2/(2*sigma_x**2) -(y-y0)**2/(2*sigma_y**2))

### curve_fit only takes in 1D data, so here is a flattened version of the gaussian

In [ ]:
def twoD_Gaussian(xy, A, x0, y0, sigma_x, sigma_y):         ### This is actually not used, the function in the next cell is what is used to fit the point sources 
    x, y = xy
    g = A*np.exp(-(x-x0)**2/(2*sigma_x**2) -(y-y0)**2/(2*sigma_y**2))
    return g.ravel()

### Find the necessary parameters to fit the 2D gaussian using the data set

In [ ]:
y0, x0 = np.unravel_index(np.argmax(ex_sit[0].data), ex_sit[0].data.shape)
sigma = np.std(ex_sit[0].data)
amp = np.max(ex_sit[0].data)

### Use the parameters above and the functional form to fit the gaussian with curve_fit

In [ ]:
init_guess = (amp, x0, y0, 0.5, 0.5)

fit, fit_var = curve_fit(twoD_Gaussian, (xi_sit, yi_sit), ex_sit[0].data.ravel(), p0=init_guess)

data_fit = twoD_Gaussian((xi_sit, yi_sit), fit[0], fit[1], fit[2], fit[3], fit[4])
data_fit = data_fit.reshape(ex_sit[0].shape)
fit

### Plot the model

In [ ]:
fig = plt.figure()
frame = fig.add_subplot(1,1,1, projection='3d', azim=-45, elev=30)
frame.plot_surface(xi_sit, yi_sit, data_fit, cmap='jet')
frame.set_xlabel('X-pixel')
frame.set_ylabel('Y-pixel')
frame.set_zlabel('Flux Amplitude')
frame.set_title(f'3D SITELLE fitted star in {galaxy}')
plt.show()

### Plot the SITELLE stars

In [ ]:
fig = plt.figure(figsize=(7, 7))
axes = [plt.subplot(4,5,i+1) for i in range(len(ex_sit))]

for i in range(len(axes)):
    y0, x0 = np.unravel_index(np.argmax(ex_sit[i].data), ex_sit[i].data.shape)
    yi_sit, xi_sit = np.indices(ex_sit[i].data.shape)
    amp = np.max(ex_sit[i].data)
    init_guess = (amp, x0, y0, 0.5, 0.5)
    fit, fit_var = curve_fit(twoD_Gaussian, (xi_sit, yi_sit), ex_sit[i].data.ravel(), p0=init_guess)
    fwhm = 2.355 * np.mean([fit[3], fit[4]]) * 0.321

    norm = simple_norm(ex_sit[i].data, 'log', percent=99.0)
    axes[i].imshow(ex_sit[i],  origin='lower', cmap='viridis', norm=norm)#, vmin=np.quantile(ex_sit[i].data, 0.01), vmax=np.quantile(ex_sit[i].data, 0.9))

    axes[i].set_title(f'FWHM: {round(fwhm, 4)}"', fontsize=9)

plt.tight_layout()
fig.suptitle(f'SITELLE stars in {galaxy}', y=1.02)#, fontsize=20)

### Plot the MUSE stars

In [ ]:
fig = plt.figure(figsize=(7, 7))
axes = [plt.subplot(4,2,i+1) for i in range(len(ex_muse))]

for i in range(len(axes)):
    y0, x0 = np.unravel_index(np.argmax(ex_muse[i].data), ex_muse[i].data.shape)
    yi_muse, xi_muse = np.indices(ex_muse[i].data.shape)
    amp = np.max(ex_muse[i].data)
    init_guess = (amp, x0, y0, 0.5, 0.5)
    fit, fit_var = curve_fit(twoD_Gaussian, (xi_muse, yi_muse), ex_muse[i].data.ravel(), p0=init_guess)
    fwhm = 2.355 * np.mean([fit[3], fit[4]]) * 0.2

    norm = simple_norm(ex_muse[i].data, 'log', percent=99.0)
    axes[i].imshow(ex_muse[i],  origin='lower', cmap='viridis', norm=norm)#, vmin=np.quantile(ex_muse[i].data, 0.01), vmax=np.quantile(ex_muse[i].data, 0.9))
    
    axes[i].set_title(f'FWHM: {round(fwhm, 4)}"', fontsize=9)
    #axes[i].set_title(f'{muse_bright_ind[i]}', fontsize=9)
    #axes[i].set_title(f'{i}', fontsize=9)

plt.tight_layout()
fig.suptitle(f'MUSE stars in {galaxy}', y=1.02, x=0.25)#, fontsize=20)

### Make the ePSF objects for both images

In [ ]:
oversamp = 3
epsf_builder = EPSFBuilder(oversampling=oversamp, maxiters=7, smoothing_kernel= 'quadratic',
                            progress_bar=False) 
epsf_sit, fitted_stars_sit = epsf_builder(ex_sit)
epsf_muse, fitted_stars_muse = epsf_builder(ex_muse)

### Plot an ePSf from each image for comparison

In [ ]:
nrows = 1
ncols = 2
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 5), squeeze=True)
ax = ax.ravel()

norm = simple_norm(epsf_sit.data, 'log', percent=99.0)
ax[0].imshow(epsf_sit.data,  origin='lower', cmap='viridis', norm=norm)#, vmin=np.quantile(epsf_sit.data, 0.1), vmax=np.quantile(epsf_sit.data, 0.99))

y0, x0 = np.unravel_index(np.argmax(epsf_sit.data), epsf_sit.data.shape)
amp = np.max(epsf_sit.data)
init_guess = (amp, x0, y0, 0.5, 0.5)
yi, xi = np.indices(epsf_sit.data.shape)
fit, fit_var = curve_fit(twoD_Gaussian, (xi, yi), epsf_sit.data.ravel(), p0=init_guess)
fwhm = 2.355 * np.mean([fit[3], fit[4]]) * 0.321
fwhm /= oversamp    ### Account for oversampling when calculating fwhm

ax[0].set_title(f'SITELLE {galaxy} ePSF, FWHM: {round(fwhm, 4)}')

norm = simple_norm(epsf_muse.data, 'log', percent=99.0)
ax[1].imshow(epsf_muse.data, origin='lower', cmap='viridis', norm=norm)#, vmin=np.quantile(epsf_muse.data, 0.1), vmax=np.quantile(epsf_muse.data, 0.99))

y0, x0 = np.unravel_index(np.argmax(epsf_muse.data), epsf_muse.data.shape)
amp = np.max(epsf_muse.data)
init_guess = (amp, x0, y0, 0.5, 0.5)
yi, xi = np.indices(epsf_muse.data.shape)
fit, fit_var = curve_fit(twoD_Gaussian, (xi, yi), epsf_muse.data.ravel(), p0=init_guess)
fwhm = 2.355 * np.mean([fit[3], fit[4]]) * 0.2
fwhm /= oversamp   ### Account for oversampling when calculating fwhm
ax[1].set_title(f'MUSE {galaxy} ePSF, FWHM: {round(fwhm, 4)}')

### Plot the input SITELLE stars with the updated centers and fluxes

In [ ]:
fig = plt.figure(figsize=(7, 7))
axes = [plt.subplot(4,4,i+1) for i in range(len(ex_sit))]

for i in range(len(axes)):
    axes[i].imshow(fitted_stars_sit.data[i],  origin='lower', cmap='gist_heat', vmin=np.quantile(fitted_stars_sit.data[i], 0.01), vmax=np.quantile(fitted_stars_sit.data[i], 0.9995))

plt.tight_layout()
fig.suptitle(f'SITELLE ePSF stars in {galaxy}', y=1.02, fontsize=10)

### Plot the input MUSE stars with the updated centers and fluxes

In [ ]:
fig = plt.figure(figsize=(7, 7))
axes = [plt.subplot(4,4,i+1) for i in range(len(ex_muse))]

for i in range(len(axes)):
    axes[i].imshow(fitted_stars_muse.data[i],  origin='lower', cmap='gist_heat', vmin=np.quantile(fitted_stars_muse.data[i], 0.01), vmax=np.quantile(fitted_stars_muse.data[i], 0.995))

plt.tight_layout()
fig.suptitle(f'MUSE ePSF stars in {galaxy}', y=1.02, fontsize=10)

### Plot the MUSE deep frame image with the ePSF fitted locations

In [ ]:
gc = aplpy.FITSFigure(fits_rband)

plt.scatter(np.transpose(fitted_stars_muse.center_flat)[0], np.transpose(fitted_stars_muse.center_flat)[1], marker='o', facecolors='none', linewidth=2, s=100, edgecolors='blue', alpha=1)

gc.recenter(coord_dic[galaxy][0], coord_dic[galaxy][1], width=zoom_dic[galaxy][0], height=zoom_dic[galaxy][1])
gc.show_colorscale(cmap='gist_heat', vmin=np.quantile(fits_rband.data, 0.01), vmax=np.quantile(fits_rband.data, 0.995))
gc.set_title(f'MUSE deep frame for {galaxy}')
gc.add_grid()

### Plot the SITELLE image with the ePSF fitted locations

In [ ]:
gc = aplpy.FITSFigure(sit_deep)

plt.scatter(np.transpose(fitted_stars_sit.center_flat)[0], np.transpose(fitted_stars_sit.center_flat)[1], marker='o', facecolors='none', linewidth=2, s=100, edgecolors='blue', alpha=1)

#gc.recenter(coord_dic[galaxy][0], coord_dic[galaxy][1], width=zoom_dic[galaxy][0], height=zoom_dic[galaxy][0])
gc.show_colorscale(cmap='gist_heat', vmin=np.quantile(sit_deep.data, 0.1), vmax=np.quantile(sit_deep.data, 0.9995))
gc.set_title(f'SITELLE deep frame for {galaxy}')
gc.add_grid()